# Linear discriminant of damage

This notebook explores the relationship between reported damage and a combination of wind and rainfall as hazard parameters. Consideration is given to using linear (or quadratic) discriminant analysis (e.g. Wilks, 1999) to determine thersholds for damage/no damage.


In [1]:
%matplotlib inline

import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import seaborn as sns


In [6]:
root_path = "//prod.lan/active/ops/community_safety/georisk/"
data_path = r"\\prod.lan\active\ops\community_safety/georisk/HaRIA_B_Wind/projects/impact_forecasting/data/exposure/NSW/April_2015_Impact_Assessment"
#data_path = "C:/WorkSpace/data/derived/exposure/NSW"
filename = "Property_damage_cleaned.shp" 
filepath = pjoin( data_path, filename)

gdf = gpd.read_file(filepath)

DMG_ORDER=['No Damage - 0%','Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%','Destroyed - 76-100%']


We start with a simple scatter plot of the (dependent) categorical damage state against the independent variables (the forecast variables). Different damage states are coloured differently. There's clearly a lot of scatter and no discernable pattern here. 

In [9]:
gdf.columns

Index(['OBJECTID', 'createdate', 'GlobalID', 'srcglobali', 'papoid', 'srcid',
       'assessoff', 'teamid', 'dateinsp', 'agency', 'eventname', 'eventtype',
       'areainspd', 'position_', 'number_', 'street', 'streettype', 'suburb',
       'state_', 'postcode', 'country', 'longaddrss', 'gurasaddrs', 'latitude',
       'longitude', 'propinsurd', 'continsurd', 'buildnguse', 'foundation',
       'structure_', 'roof', 'floorlngth', 'floorlng_1', 'floorwidth',
       'numbsement', 'numaprt', 'numaprthab', 'asbestos', 'pwrwiredwn',
       'dangertree', 'othdebris', 'Field42', 'propstatus', 'degdamage',
       'roofdam', 'frmecollap', 'bldleaning', 'rackingdam', 'grndmovemt',
       'Field50', 'waterinund', 'maxwatrlev', 'washoutreq', 'validaddre',
       'EICU_Degda', 'EICU_Deg_1', 'Comments', 'geometry'],
      dtype='object')

In [8]:
sns.lmplot(x='PSWG', y='P1RR',hue='EICU_Degda', data=gdf, fit_reg=False, aspect=1.5,hue_order=DMG_ORDER)

KeyError: "['P1RR', 'PSWG'] not in index"

So let's group all damaged buildings into a single category, irrsepective of the level of damage. We add a new field to the damage state and assign it a value of 0 (not damaged) or 1 (damaged).

In [ ]:
damaged = np.zeros(len(gdf))
damaged[gdf['EICU_Degda'].isin(['Destroyed - 76-100%', 
                                 'Severe Impact - 51-75%', 
                                 'Major Impact - 26-50%', 
                                 ])] = 1

gdf['Damaged'] = damaged

sns.lmplot(x='maxwind', y='rainrate',hue='Damaged', data=gdf, fit_reg=False, aspect=1.5,)

In [ ]:
X = np.array([gdf['maxwind'].values, gdf['rainrate'].values]).T
y = damaged
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from matplotlib import colors
clf = QuadraticDiscriminantAnalysis()
clf.fit(X, y)

In [ ]:
fig, ax = plt.subplots(1,1)
ypred = clf.predict(X)
Z = clf.predict_proba(X)
cm = ax.scatter(X[:,0], X[:,1], c=Z[:,1], cmap='RdBu')
plt.colorbar(cm)
plt.xlabel("Maximum wind gust (m/s)")
plt.ylabel("Maximum rainfall rate (mm/hr)")

We now plot the results of the QDA as a contour surface, so we can see what happens towards the extrema of the values. 

In [ ]:
xx = np.linspace(0, 15, 1000)
yy = np.linspace(0, 160, 1000)
xp, yp = np.meshgrid(xx, yy)
Z = clf.predict_proba(np.c_[xp.ravel(), yp.ravel()])
Z = Z[:, 1].reshape(xp.shape)
fig, ax = plt.subplots(1,1)
cm = plt.pcolormesh(xp, yp, Z, cmap='cubehelix_r', norm=colors.Normalize(0., 1.))
cs = plt.contour(xp, yp, Z, [0.05, 0.1, 0.25,0.5, 0.75], linewidths=2., colors='k')
plt.clabel(cs, fmt='%.2f')
plt.colorbar(cm, label="Probability of damage")
plt.xlabel("Maximum wind gust (m/s)")
plt.ylabel("Maximum rainfall rate (mm/hr)")
plt.title("Probability of damage")

Using a quadratic discriminant shows somewhat better realism, with decreasing probability for low wind speeds across the range of values of rainfall rate. But the lack of observations of high wind speed adversely affects the expected behaviour above about 8 m/s, where the probability of damage falls away with increasing wind speed.

The only real solution in this situation is more observations. 

### More categories?

We can extend this analysis to more categories. Assigning a value to the different damage states and performing a similar analysis means we can explore the probability of different damage states for a given pairing of incident wind speed and maximum rainfall rate. 

In [ ]:
damaged = np.zeros(len(gdf))
damaged[gdf['EICU_Degda'].isin(['Major Impact - 26-50%'])] = 1
damaged[gdf['EICU_Degda'].isin(['Severe Impact - 51-75%'])] = 2
damaged[gdf['EICU_Degda'].isin(['Destroyed - 76-100%'])] = 3, 


gdf['Damaged'] = damaged

sns.lmplot(x='maxwind', y='rainrate',hue='Damaged', data=gdf, fit_reg=False, aspect=1.5,)

We perform the QDA in the same manner. The resulting fit will give a probability of each damage state, which we can plot as a contour set. 

In [ ]:
X = np.array([gdf['maxwind'].values, gdf['rainrate'].values]).T
y = damaged
clf = QuadraticDiscriminantAnalysis()
clf.fit(X, y)

In [ ]:
xx = np.linspace(0, 15, 1000)
yy = np.linspace(0, 160, 1000)
xp, yp = np.meshgrid(xx, yy)
Z = clf.predict_proba(np.c_[xp.ravel(), yp.ravel()])
fig, axes = plt.subplots(2,2, sharex=True, sharey=True, figsize=(10, 10))
dmg_states = ['No', 'Major', 'Severe', 'Complete']
for i, ax in enumerate(axes.flatten()):
    s = Z[:, i].reshape(xp.shape)
    cm = ax.pcolormesh(xp, yp, s, cmap='cubehelix_r', norm=colors.Normalize(0., 1.))
    cs = ax.contour(xp, yp, s, [0.01, 0.05, 0.1, 0.25,0.5, 0.75], linewidths=2., colors='k')
    ax.clabel(cs, fmt='%.2f')
    #plt.colorbar(cm, label="Probability of {0} damage".format(dmg_states[i]))
    ax.set_xlabel("Maximum wind gust (m/s)")
    ax.set_ylabel("Maximum rainfall rate (mm/hr)")
    ax.set_title("{0} damage".format(dmg_states[i]))

    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([1.1, 0.15, 0.05, 0.7])
fig.colorbar(cm, cax=cbar_ax, label="Probability of damage state")
fig.tight_layout()

In [ ]:
clf.predict_proba(X)

In [ ]:
f, axes = plt.subplots(2,3,sharex=True, sharey=True)

for ax, dmgstate in zip(axes.flat, DMG_ORDER[0:]):
    df = gdf[gdf['EICU_Degda']==dmgstate][['maxwind', 'rainrate']]
    sns.kdeplot(df['maxwind'], df['rainrate'], ax=ax)
    n = len(df)
    ax.set_title("{0} (n={1})".format(dmgstate, n), fontsize='small')
    ax.set_xlim((0, 20))
    ax.set_ylim((0, 160))
f.tight_layout()

So it looks like adding maximum rainfall rate (kg/m2/s) as a hazard parameter will not improve the discrimination of damage states. The peak density for the destroyed buildings (n=8) overlaps with the no damage density (n=1465), and is almost colocated with the peak for minor impact.

Another option would be to explore accumulated rainfall amounts.

In [ ]:
gdf['roofdam'].unique()

In [ ]:
ROOF_DMG_ORDER = ['Stable','Minor','Moderate','Severe']
sns.lmplot(x='maxwind', y='rainrate',hue='roofdam', data=gdf, fit_reg=False, aspect=1,hue_order=ROOF_DMG_ORDER)

In [ ]:
f, axes = plt.subplots(2,2,sharex=True, sharey=True)

for ax, dmgstate in zip(axes.flat, ROOF_DMG_ORDER):
    df = gdf[gdf['roofdam']==dmgstate][['maxwind', 'rainrate']]
    n = len(df)
    sns.jointplot(df['maxwind'], df['rainrate'],kind='kde', ax=ax)
    ax.set_title("{0} (n={1})".format(dmgstate, n))
    ax.set_xlim((0, 20))
    ax.set_ylim((0, 150))
f.tight_layout()

In [ ]:
gdf.groupby([ 'waterinund','roofdam','EICU_Degda']).agg('mean')['rainrate']

In [ ]:
gdf.groupby([ 'waterinund','roofdam','EICU_Degda']).agg('mean')['maxwind']

In [ ]:
gdf['EICU_Degda'].isin(['Severe Impact - 51-75%','Major Impact - 26-50%','Destroyed - 76-100%'])